# Trigger one or several cases within a Dayahead workflow

1. Define your Dayahead workflow
2. Connect your workflow with a powerops client
3. Trigger your Dayahead workflow

### Import the DayaheadTrigger class and define your workflow
Define the Dayahead configuration you want to run via the DayaheadTrigger class in the sdk. The following code snippet shows how this can be achieved. You will need to define what price scenarios to run, the price area, the method, and reference the prerun files by external ID in CDF (this means that the files must exist in CDF)

In [38]:
import yaml
from cognite.powerops.client.shop.data_classes.dayahead_trigger import DayaheadTrigger


dayahead_dict = yaml.safe_load(
        """
        price_scenarios:
        - "scenario_1"
        - "scenario_2"
        price_area: "NO2"
        method: multi_scenario_10
        cases:
        - case_name: Fornebu
          pre_runs:
          - pre_run_external_id: "Prerun_Fornebu_multi_scenario_10_1.yaml"
          - pre_run_external_id: "Prerun_Fornebu_multi_scenario_10_2.yaml"
        """
)

dayahead = DayaheadTrigger(**dayahead_dict)

### Trigger the workflow with the powerops client
Instantiate a powerops client and use the DayaheadTrigger object previously created to instantiate a DayaheadTriggerAPI class. Then call trigger function to create the necessary resources in CDF and trigger CogShop with these SHOP runs

In [26]:
# You can control which setting files are loaded through the environmental variable below.
# In this case, the setting files are located two levels above, in the root of the repository.
import os
from cognite.powerops import PowerOpsClient
from cognite.powerops.client.shop.api.dayahead_trigger_api import DayaheadTriggerAPI

SHOP_DATASET_ID = 4272372867810023

os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

trigger_dayahead = DayaheadTriggerAPI(client=powerops.cdf,
                                     data_set=SHOP_DATASET_ID)

trigger_event = powerops.workflow.trigger_workflow(dayahead)

In [27]:
trigger_event

{'workflow_trigger_event': 'POWEROPS_BID_PROCESS_multi_scenario_10_2_NO2_def3b5',
 'shop_run_events': ['SHOP_RUN_2023-11-15T13:43:48Z_7b1ffd',
  'SHOP_RUN_2023-11-15T13:43:48Z_30488f']}

### Outcome of triggered case
The trigger function creates a shop run event for each reference to a prerun file. These are the "SHOP_RUN_" events. as well as a  “binding” triggering_event that gets a relationship to each shop run event. 
The plan is to use this triggering event to pass to the total_bid_matrix calculation later on with the necessary information to run that step

## Explore each of the triggered SHOPRuns

`SHOPRun` represents a single attempt at running SHOP.

In [28]:
# Retrieve an existing shop run by event external_id.
shop_run_0 = powerops.shop.retrieve("SHOP_RUN_2023-11-15T13:43:48Z_7b1ffd")
shop_run_0.check_status()

<SHOPRunStatus.FAILED: 'failed'>

In [29]:
print(shop_run_0)

{
    "external_id": "SHOP_RUN_2023-11-15T13:43:48Z_7b1ffd",
    "watercourse": "Fornebu",
    "start": "2023-11-15 13:43:48+00:00",
    "end": null,
    "case_file_external_id": "Prerun_Fornebu_multi_scenario_10_1.yaml",
    "shop_files_external_ids": [
        {
            "external_id": "SHOP_Fornebu_commands",
            "file_type": "ascii"
        }
    ],
    "shop_version": "CogShop2-20230614T102157Z"
}


In [30]:
# Small helper to display first 10 lines of a long string (e.g. content of a file).
from IPython.display import display, HTML

def preview_content(content):
    N_LINES = 10
    lines = content.splitlines()
    trimmed_content = "\n".join(lines[:N_LINES])
    if len(lines) > N_LINES:
        trimmed_content += "\n..."
    display(HTML(f"<pre style='font-size: 75%; border: 1px solid var(--jp-content-font-color1); padding: 10px; margin: 10px 0 0 10px;'>{trimmed_content.strip()}</pre>"))

# Same, just for a file.
def preview_file(file_path):
    with open(file_path) as file:
        preview_content(file.read())

In [31]:
# Show first 10 lines of each additional SHOP file in the case.
N_LINES = 10
for shop_file_content in shop_run_0.get_shop_files():
    preview_content(shop_file_content)

# also available:
# shop_run_0.get_case_file()  # content of the case file
# list(shop_run_0.get_log_files())  # content of log files, generated by SHOP